# ML Flow on Azure ML

The ML ops demo notebook shows running ML Flow on a local machine, and the AzureML notebook demonstrates using the Azure ML SDK for experiment tracking. This notebook combines the two, using AzureML to run, but tracking through the ML Flow API with AzureML providing the backend storage. This allows us to make use of the easily scaling  infrastructure of AzureML, while the code is still portable as other backends can easily be swapped in when required.


In [1]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
%load_ext tensorboard

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv1D, concatenate
from tensorflow.keras.layers import ZeroPadding1D, Reshape, Input, Dropout, PReLU
from tensorflow.keras.models import Sequential, Model

from sklearn.metrics import mean_absolute_error, r2_score

In [5]:
import mlflow
mlflow.tensorflow.autolog()

2022/08/19 12:14:49 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


In [7]:
import azureml
import azureml.core

In [6]:
import prd_pipeline

In [16]:
import importlib

In [23]:
importlib.reload(prd_pipeline)

<module 'prd_pipeline' from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/prd-ml-pipeline/code/Users/stephen.haddad/precip_rediagnosis/model_pipeline/prd_pipeline.py'>

### Load data

In [8]:
train202208_datastore_name = 'precip_rediagnosis_train202208'
prd_merged_file_dataset_name = 'prd_merged_csv_files'
prd_prefix = 'prd'
merged_prefix = prd_prefix + '_merged'
csv_file_suffix = 'csv'

In [9]:
azure_experiment_name='prd_mlops_test'
azure_env_name = 'prd_ml_cluster'
cluster_name = 'mlops-test'

In [10]:
prd_model_name = 'azml_mlops_202208'

In [20]:
prd_all_events_dataset_name = 'prd_merged_all_events_files'

In [11]:
target_parameter = 'rainfall_rate_composite'
profile_features = ['air_temperature', 'relative_humidity']
single_lvl_features = ['air_pressure_at_sea_level'] 

In [13]:
prd_ws = azureml.core.Workspace.from_config()

In [14]:
mlflow.set_tracking_uri(prd_ws.get_mlflow_tracking_uri())

In [24]:
input_data = prd_pipeline.load_data(
    current_ws=prd_ws,
    dataset_name=prd_all_events_dataset_name
)
data_splits, data_dims = prd_pipeline.preprocess_data(
    input_data=input_data,
    test_fraction=0.2,
    feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
)

loading all event data
Downloaded path: /tmp/tmpzje0kqgx/b02f8326-b2e0-4017-ac7b-be56ac5a4259/prd/202002_storm_ciara/prd_merged_20200207T1800Z_20200210T1800Z.csv is different from target path: /tmp/tmpzje0kqgx/b02f8326-b2e0-4017-ac7b-be56ac5a4259/202002_storm_ciara/prd_merged_20200207T1800Z_20200210T1800Z.csv
Downloaded path: /tmp/tmpzje0kqgx/b02f8326-b2e0-4017-ac7b-be56ac5a4259/prd/202002_storm_dennis/prd_merged_20200214T1800Z_20200217T1800Z.csv is different from target path: /tmp/tmpzje0kqgx/b02f8326-b2e0-4017-ac7b-be56ac5a4259/202002_storm_dennis/prd_merged_20200214T1800Z_20200217T1800Z.csv
Downloaded path: /tmp/tmpzje0kqgx/b02f8326-b2e0-4017-ac7b-be56ac5a4259/prd/202008_storm_ellen/prd_merged_20200819T0600Z_20200822T1200Z.csv is different from target path: /tmp/tmpzje0kqgx/b02f8326-b2e0-4017-ac7b-be56ac5a4259/202008_storm_ellen/prd_merged_20200819T0600Z_20200822T1200Z.csv
Downloaded path: /tmp/tmpzje0kqgx/b02f8326-b2e0-4017-ac7b-be56ac5a4259/prd/202008_storm_francis/prd_merged_2020

/mnt/batch/tasks/shared/LS_root/mounts/clusters/prd-ml-pipeline/code/Users/stephen.haddad/precip_rediagnosis/model_pipeline/prd_pipeline.py:104: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  merged_df = pandas.concat([pandas.read_csv(p1) for p1 in prd_path_list])


target has dims: 23
dropping zeros


KeyError: 'rainfall_rate_composite'

In [ ]:
nprof_features = data_dims['nprof_features'] 
nheights = data_dims['nheights']
nsinglvl_features = data_dims['nsinglvl_features']

In [ ]:
exp1 = mlflow.create_experiment('prd_exp_azml_mlflow')
exp1

In [ ]:
exp1 = mlflow.get_experiment(exp1)

In [ ]:
exp1

In [ ]:
import datetime
log_dir = 'log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [ ]:
tensorflow_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# run tensorboard --logdir LOGDIRPATH from command line to launch 

In [ ]:
import tempfile

In [ ]:
with mlflow.start_run(experiment_id=exp1.experiment_id) as current_run:
    model = prd_pipeline.build_model(nprof_features, nheights, nsinglvl_features)
    model.summary()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss='mean_absolute_error', optimizer=optimizer)

    history = model.fit(data_splits['X_train'], data_splits['y_train'], epochs=50, batch_size=128, validation_data=(data_splits['X_val'], data_splits['y_val']), verbose=True, callbacks=[tensorflow_callback])

    y_pred = model.predict(data_splits['X_val'])
    error = mean_absolute_error(data_splits['y_val'], y_pred)
    print(f'MAE: {error:.3f}')
    rsqrd = r2_score(data_splits['y_val'], y_pred)
    print(f'R-squared score: {rsqrd:.3f}')
    
    mlflow.log_metric('MAE', error)
    mlflow.log_metric('R-squared', rsqrd)
    
    fig1 = plt.figure(figsize=(10, 8))
    ax1 = fig1.add_subplot(1,1,1)
    ax1.scatter(data_splits['y_val'], y_pred, s=200, c='darkblue')
    ax1.plot([0, 300], [0, 300], ls="--", c=".3")
    ax1.set_xlabel('Actual 3hr precip accumulation value')
    ax1.set_ylabel('Predicted 3hr precip_accumulation value')
#     with tempfile.TemporaryDirectory() as td1:
        
#         fig1.savefig(plot_out_path, bbox_inches='tight')
    mlflow.log_figure(fig1,  'actual_predicted_precip_3hr.png')                   
    # mlflow.log


If we look at the experiment in AzureML GUI, we see that all the model parameters have been automatically logged, and the model has been saved by ML Flow ready for use in inference.

## Evaluation

In [ ]:
training_hist_df = pd.DataFrame(history.history)
training_hist_df['epoch'] = history.epoch

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(training_hist_df.epoch, training_hist_df.loss, label='training')
plt.plot(training_hist_df.epoch, training_hist_df.val_loss, c='g', label='validation')
plt.legend()
plt.ylabel('MAE [mm of precipitation]')
plt.xlabel('epochs')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(data_splits['y_val'], alpha=0.5, bins=40, label='Actual')
plt.hist(y_pred, alpha=0.5, bins=40, label='Predicted')
plt.legend()
plt.show()